# Logique combinatoire - PLD - SUJET


**Ressources** : <button><a style="text-decoration:none" href="https://webge.fr/dokuwiki/doku.php?id=materiels:pld#hardware_-_pld_cpld">PLD, CPLD sur WebGE</a></button>
<button><a style="text-decoration:none" href="https://webge.fr/dokuwiki/doku.php?id=python:accueilpython" target="_blank">Wiki Python sur WebGE</a></button>
<button><a style="text-decoration:none" href="https://webge.fr/doc/tnsi/tp/Architectures/TNSI_ARCH_DR_Logique_Comb_PLD.zip" target="_blank">Document réponse</a></button>
<button><a style="text-decoration:none" href="https://webge.fr/doc/tnsi/tp/Architectures/AidecreerVecteursTest.pdf" target="_blank">Aide creerVecteursTest</a></button>

---

**Sommaire**
<ol>
    <li>Présentation
        <ol>
            <li>Historique</li>
            <li>La fonction "décodage"</li>
        </ol>
    </li>
    <li>Enoncé</li>
    <li>Equations de la fonction "Décodage"</li>
    <li>Simulation
        <ol>
            <li>Préparation des vecteurs de test</li>
            <li>Exemple d'utilisation des vecteurs de test</li>
            <li>Test des sorties RAM, ROM et OE de la fonction "Décodage"</li>
            <li>Génération automatique de la table des vecteurs de test</li>
        </ol>
    </li>
    <li>Synthèse dans WokWi : testeur de circuits logiques</li>
</ol>

----

## 1. Présentation
Les circuits électroniques (microprocesseur, chipset, System on Chip) assurent le fonctionnement de nos outils numériques (ordinateur, tablette, smartphone, etc.). Ils sont constitués de fonctions logiques complexes (unité arithmétique et logique, multiplexeur, décodeur, etc.) issues de l'assemblage d'opérateurs logiques <strong>combinatoires</strong> (ET, OU, NON, etc.) et séquentiels (bascules). 

Dans ce TP, vous allez **simuler** les tests à effectuer sur une fonction logique avant son intégration  dans un circuit logique programmable.

### 1.A Historique

<table >
    <tr>
 <td style="text-align:justify; font-size:14px">
            <p>La figure ci-contre représente le boîtier <strong>DIP</strong> (<strong>D</strong>ual <strong>I</strong>n <strong>P</strong>ackage) à 14 broches d’un circuit intégré de type <strong><a href="https://fr.wikipedia.org/wiki/Circuit_int%C3%A9gr%C3%A9" target="_blank">SSI</a></strong> (<strong>S</strong>mall <strong>S</strong>cale <strong>I</strong>ntegration) contenant quatre portes ET à deux entrées. <br> Apparus en 1964, les circuits logiques SSI sont encore utilisés. Ils contiennent de 1 à 12 portes. Dans les <strong>années 80</strong>, une fonction logique telle que celle représentée par le logigramme ci-dessous nécessitait l’utilisation de <strong>trois circuits intégrés </strong> de type SSI intégrant chacun quatre portes logiques.</p> <br> Ces composants étaient soudés et interconnectés par des pistes de cuivre sur un circuit imprimé.
        </td>
        <td>
            <img src="img/7400.png">
        </td>
    </tr>
</table>

<div style="text-align: center" ><img src ="img/decodeur.jpg"></div>.

<table >
    <tr>
        <td>
            <div style="text-align:center"><img src="img/fpga.png"></div>
        </td>
        <td style="text-align:justify; font-size:14px">
            On dispose <strong>aujourd’hui</strong> de <strong>circuits logiques programmables</strong> (<strong>P</strong>rogrammable <strong>L</strong>ogic <strong>D</strong>evices) capables, selon leur complexité, de remplacer quelques dizaines à plusieurs centaines de milliers de circuits intégrés. Le mot "programmable" ne signifie pas que le circuit exécute un programme, mais que les opérateurs logiques qu'il contient sont interconnectés à l'aide d'un programme ! Si la fonction réalisée ne correspond pas à celle attendue ou si on souhaite la faire évoluer, le composant pourra être "effacé" et reprogrammé. <a href="https://webge.fr/dokuwiki/doku.php?id=materiels:pld" target="_blank">Pour en savoir plus</a>.<br><br>
            Un circuit logique programmable renferme un grand nombre d’<strong>opérateurs logiques combinatoires</strong> (<strong>ET, OU, NON, etc</strong>.) et séquentiels (bascules). Ce type de composant est utilisé lors de la <strong>conception ou lors de la réorganisation</strong> d’un produit nécessitant la mise en œuvre d’un grand nombre d’opérateurs logiques, car il améliore la fiabilité du produit, diminue son coût de fabrication et lui confère une certaine évolutivité. <a href="https://fr.wikipedia.org/wiki/Circuit_logique_programmable" target="_blank">Wikipédia</a>.
        </td>
    </tr>
</table>

<table >
    <tr>
        <td style="text-align:justify; font-size:14px">
            L’<strong>interconnexion</strong> des opérateurs logiques est réalisée avec un <strong>outil de programmation</strong> (ordinateur + logiciel).<br><br>
Lorsque la fonction à intégrer dans le PLD est <strong>combinatoire</strong> et suffisamment simple, il suffit d'écrire les <strong>équations logiques booléennes de ses sorties</strong> avec le logiciel de programmation. 
        </td>
        <td>
            <div style="text-align:center"><img src="img/quartus.jpg" width="200px"></div>
        </td>
    </tr>
</table>

<table >
    <tr>
        <td><div style="text-align:center"><img src="img/migen_logo.png"></div></td>
         <td style="text-align:justify; font-size:14px">La "programmation" d'un circuit logique programmable est réalisée avec un langage de description matériel ou  un outil de saisie graphique. Après compilation de cette description, on obtient un fichier de configuration pour le circuit choisi. <br> <strong>VHDL</strong> et <strong>Verilog</strong> sont les deux langages de description les plus répandus. Il existe cependant un outil, <a href="https://m-labs.hk/migen/manual/introduction.html" target="_blank">migen</a>, basé sur Python qui vise à automatiser davantage le processus de conception.</td>
    </tr>
</table>

### 1.B La fonction "Décodage"
La fonction logique "**Décodage**" est une fonction **combinatoire**. Elle reçoit n bits en entrée (2<sup>n</sup> combinaisons) et délivre M bits en sortie telle que <strong>M ≤ 2<sup>n</sup></strong>. La fonction "Décodage" est courante en électronique numérique. Elle est notamment utilisée dans les microprocesseurs pour réaliser le **décodeur d'instructions**.
<div style="text-align:center"><img src="img/registres.jpg"></div>

## 2. Enoncé
Dans le cadre de la modification d’une carte électronique, on souhaite intégrer une fonction **Décodage**, initialement réalisée avec trois composants logiques, dans un PLD. Le logigramme de cette fonction est donné ci-dessous.<div style="text-align:center"><img src="img/decodeur3.png"></div>

#### Logigramme
<div style="text-align:center"><img src="img/decodeur2.png"></div>div>
<p style="text-align:right;">Dessiné avec <a href="https://www.digikey.com/schemeit/" target="_blank">Scheme-it.</a></p>

## 3. Equations de la fonction "Décodage"
On limite l'étude à trois des six sorties de la fonction "**Décodage**". 

> **Activité 1**
>
> Si l'activité n'a pas été préparée en cours, **écrivez** les équations de RAM, ROM et OE sur le **document réponse**. 

## 4. Simulation
Après la phase de programmation d’un PLD, il est nécessaire d’en vérifier le fonctionnement. Dans le cas du test d’une fonction combinatoire, il suffit de placer sur ses entrées toutes les combinaisons logiques qu’elle est susceptible de recevoir et de comparer chaque état de sa sortie avec celui attendu. Ceci est fait automatiquement par l’outil de programmation. Seule l’écriture des **vecteurs de test** incombe au concepteur.

Un vecteur de test se compose des **valeurs à placer sur les entrées** (E1, E2, .., En) de la fonction à tester et du **résultat attendu** sur sa sortie (S). Le vecteur de test i peut s'exprimer sous la forme : **Vi(E1, E2, .., En, S)**

__Exemple__ : test d’une fonction f() implantée dans un PLD telle que **S = /E1.E2 + E1./E2** (/ <=> not)

<div style="text-align:center"><img src="img/extest.png" title="Fonction Ou Exclusif"></div>div>

Dans l’exemple ci-dessus, les vecteurs V1(0,0,0), V2(0,1,1), V3(1,0,1) et V4(1,1,0) sont placés successivement sur les entrées du PLD reliées à la fonction f(). 

<p style="background-color:lightgrey; text-align:center; font-size:15px;">Pour tester une sortie logique combinatoire, il est nécessaire d’écrire <strong>2<sup>n</sup></strong> vecteurs. <br> <strong>n représente le nombre d’entrées de la fonction.</strong>.</p>

### 4.A Préparation des vecteurs de test

> **Activité 2** 
>
> **Si l'activité n'a pas été préparée en cours, complétez** les tables de vérité du document réponse avec les **résultats attendus** sur les sorties **RAM**, **ROM** et **OE**.

### 4.B Exemple d'utilisation des vecteurs de test
 * **Ressources** : <a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:listes" target="_blank">listes</a>, <a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:tuples" target="_blank">tuples</a>, <a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:controle#les_tests_ou_structures_alternatives" target="_blank">tests</a>, <a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:controle#la_boucle_for" target="_blank">boucle for...</a> et <a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:chaines#formatage" target="_blank">f-strings</a> en Python.

Le langage Python dispose des opérateurs logiques **and**, **or** et **not** pour écrire des équations logiques. Les états logiques peuvent être représentés par des 1 et des 0. 

**0** <=>False, **1** <=> True. 

L'exemple ci-dessous illustre le test d'un opérateur **Ou Exclusif** à 2 entrées.

In [ ]:
# Exemple introductif
'''
Test d'une porte OU Exclusif à 2 entrées
E1, E2 : entrées
S : sortie (Sa est la sortie attendue, So est la sortie obtenue)
'''
Vecteurs_XOR = [(0, 0, 0), (0, 1, 1), (1, 0, 1), (1, 1, 0)] # Une liste de tuples matérialise la table des vecteurs de test
print("[E1,E2] -> S ")
for E1, E2, Sa in Vecteurs_XOR:
    So = (not(E1) and E2 or E1 and not(E2)) # Equation de la porte OU Exclusif, parenthèses INDISPENSABLES !
    if Sa == So: 
        print(f"[{E1},{E2}] -> {Sa}") # Afficher "Sa" garantie d'avoir uniquement des 0 et des 1 dans la table !
    else:
        print(f"[{E1},{E2}] -> Erreur")

>**Activité 3. Simulons des erreurs de frappe !** <br>
>
> **Testez** les situations suivantes : 
> - le vecteur (0,1,1) a été remplacé par (0,1,0) <br>
> - l'opérateur or a été remplacé par un and <br>
>
> a) Comment identifie-t-on une erreur dans la table ? <br>
> b) Quelles sont les causes d'erreur lors du test d'un circuit logique programmable réel ? 
>
> Répondez ci-dessous.

> <span style="color:blue;">**Réponse** :</span> ???
>
>

>**Activité 4. Amélioration de l'affichage dans l'exemple introductif** <br>
>
> En reprenant l'exemple introductif, **complétez** le code ci-dessous pour que le résultat du test affiche Sa, So et précise la position des erreurs.<br>
>
> Exemple de résultat attendu <br>
> [E1,E2] -> Sa So <br>
> [0,0] -> 0,0 <br>
> [0,1] -> 0,1->Erreur <br>
> [1,0] -> 1,1 <br>
> [1,1] -> 0,0

In [ ]:
# Exemple introductif modifié
# Code de test d'une fonction XOR
'''
Test d'une porte OU Exclusif à 2 entrées
E1, E2 : entrées
S : sortie (on note Sa la sortie attendue et So est la sortie obtenue !)
'''
Vecteurs_XOR = [(0, 0, 0), (0, 1, 1), (1, 0, 1), (1, 1, 0)] # Une liste de tuples matérialise la table des vecteurs de test
# A compléter

### 4.C Test des sorties  RAM, ROM et OE de la fonction "Décodage"

> **Activité 5**
>
> En vous inspirant de l'exemple introductif modifié et en utilisant les résultats des activités 1 et 2, écrivez le code de test de la sortie **RAM**. Simulez des erreurs de frappe.

In [ ]:
# Code de test de la sortie RAM
Vecteurs_RAM = [] 
# A compléter

> **Activité 6**
>
> En vous inspirant de l'exemple introductif modifié et en utilisant les résultats des activités 1 et 2, écrivez le code de test de la sortie **ROM**. Simulez des erreurs de frappe.

In [ ]:
# Code de test de la sortie ROM
Vecteurs_ROM = []
# A compléter

> **Activité 7**
>
> En vous inspirant de l'exemple introductif modifié et en utilisant les résultats des activités 1 et 2, écrivez le code de test de la sortie **OE**. Simulez des erreurs de frappe.

In [ ]:
# Code de test de la sortie OE
Vecteurs_OE = [] # A compléter
# A compléter

### 4.D Génération automatique de la table des vecteurs de test

 * **Ressources** : 
<ul>
    <li><a href="https://webge.fr/doc/tnsi/tp/Architectures/AidecreerVecteursTest.pdf" target="_blank">Aide pour creer les Vecteurs de Test</a>
    </li>
<li><a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:listes" target="_blank">listes</a>, <a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:tuples" target="_blank">tuples</a>, <a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:controle#les_tests_ou_structures_alternatives" target="_blank">tests</a>, <a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:controle#la_boucle_for" target="_blank">boucle for...</a>, <a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:chaines#formatage" target="_blank">f-strings</a> et <a href="https://webge.fr/dokuwiki/doku.php?id=python:bases:chaines#longueur_d_une_chaine_parcours_et_selection_slice" target="_blank">sélection dans une chaîne (slicing)</a> en Python.</li></ul>

Lorsque le nombre d'entrées augmente, la table des vecteurs de test devient imposante. Pour **n=6**, il faut écrire **64 vecteurs**. L'écriture d'une telle table devient fastidieuse et le risque d'erreur augmente. Il devient alors nécessaire de disposer d'une fonction de génération automatique de la table des vecteurs de test. 

On propose l'algorithme ci-dessous.

**Remarque** : Python dispose de la fonction native <a href="https://docs.python.org/fr/3.6/library/functions.html#bin" target="_blank">bin</a>. Cette fonction convertit un nombre entier en binaire dans une chaîne avec le préfixe 0b.

In [ ]:
# Exemple 
bin(8)

>**Activité 8**. 
>
> **Complétez** la fonction *creerVecteursTest* ébauchée ci-dessous. <br> *Conseil : développer la fonction dans un éditeur (<a href="https://webge.fr/dokuwiki/doku.php?id=outils:vscode:pythonpaspas" target="_blank" title="Premiers programmes en Pyton étape par étape">VSCode</a>, etc.) avant de la tester dans le notebook.*
>
>*Résultat attendu* pour n=3 <br>
>vecteurs = [(0, 0, 0, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 1, 1), (1, 0, 0, 0), (1, 0, 1, 0), (1, 1, 0, 0), (1, 1, 1, 1)]

In [ ]:
def creerVecteursTest(n, vs):
    vecteurs = []
    # A compléter
    
    return vecteurs

# Test de la sortie Rom
Roma = [0, 1, 0, 1, 0, 0, 0, 1]
tableVecteurs = creerVecteursTest(3, Roma)
print(tableVecteurs)

> **Activité 9**.
>
> **Utilisez** la fonction *creerVecteurTest* pour tester la sortie **OE**.

In [ ]:
# Code de test de OE avec la fonction reerVecteurTest 
OEa = [] # OE attendu à compléter
Vecteurs_OE = creerVecteursTest(4, OEa)
print("[A15,A14,A13,A12] -> OEa OEo")
# A compléter

## 5. Synthèse dans WokWi : testeur de circuits logiques
Wokwi est un <b>simulateur en ligne</b>, programmable en C et en <b>microPython</b> pour les projets IdO (Internet des Objets). On peux l'utiliser pour tester le comportement des cartes Arduino, ESP32, <b>Raspberry Pi Pico</b>, etc. avec de nombreux composants standards.  

<div style="text-align:center"><img src="img/wokwi.png"></div>

<p>1. <b>Ouvrez l'exemple</b> <a href="https://wokwi.com/projects/379094333458811905" target="_blank">Testeur de CI logique</a></p>
<p>2. <b>Créez un compte</b> sur Wokwi à l'aide d'un mail (n'utilisez pas Github).</p>
<p>3. <b>Suivez les consignes</b> du fichier <b><i>Lisezmoi.txt</i></b> sur <b>Wokwi.</b>